In [14]:
import tensorflow as tf
import numpy as np
import matplotlib

In [15]:
tf.__version__, np.__version__, matplotlib.__version__

('2.16.1', '1.26.4', '3.10.0')

In [16]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Reshape
from tensorflow.keras.regularizers import L1L2
import matplotlib.pyplot as plt

In [17]:
(X_treinamento, _), (_, _) = mnist.load_data()

In [18]:
X_treinamento = X_treinamento.astype('float32') / 255

In [19]:
X_treinamento[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [20]:
X_treinamento = tf.data.Dataset.from_tensor_slices(X_treinamento).shuffle(buffer_size=60000).batch(batch_size=256)

In [21]:
type(X_treinamento)

tensorflow.python.data.ops.batch_op._BatchDataset

In [22]:
#Imagens 28x28 = 784

In [ ]:
#Gerador(recebe como entrada números aleatórios e irá criar imagens)
gerador = Sequential()
gerador.add(Dense(units=500, input_dim=100, activation = 'relu', kernel_regularizer=L1L2(1e-5,1e-5)))
#Temos 100 neurônios na camada de entrada que estão conectados com 500 neurônios da camada oculta
gerador.add(Dense(units=500, input_dim=100, activation = 'relu', kernel_regularizer=L1L2(1e-5,1e-5)))
gerador.add(Dense(units=784, activation='relu',kernel_regularizer=L1L2(1e-5,1e-5)))
#Aplicar o Reshape para transformar o formato de vetor para a imagem que é uma matriz
gerador.add(Reshape((28,28)))
gerador.summary()



Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 500)            │        50,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 784)            │       392,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 28, 28)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 693,784 (2.65 MB)

 Trainable params: 693,784 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
discriminador = Sequential()
discriminador.add(InputLayer(input_shape=(28,28)))
#Transformar os dados da imagem de matriz em vetor
discriminador.add(Flatten())
discriminador.add(Dense(units=500, activation='relu',kernel_regularizer=L1L2(1e-5,1e-5)))
discriminador.add(Dense(units=500, activation='relu',kernel_regularizer=L1L2(1e-5,1e-5)))
discriminador.add(Dense(units=1, activation='sigmoid',kernel_regularizer=L1L2(1e-5,1e-5)))
discriminador.summary()



C:\Users\nanojau\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 500)            │       392,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 500)            │       250,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 643,501 (2.45 MB)

 Trainable params: 643,501 (2.45 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
#Criar as funções que irão calcular o erro
#Saida esperada = 1
cross_entropy = tf.keras.losses.BinaryCrossentropy

In [29]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)